### Back to Instance Properties

In [ ]:
class IntegerValue:
    def __init__(self):
        self.values = {}

    def __set__(self, instance, value):
        self.values[instance] = int(value)

    def __get__(self, instance, onwer_class):
        if instance is None:
            return self
        else:
            return self.values.get(instance)

In [2]:
import weakref

In [13]:
class IntegerValue:
    def __init__(self):
        self.values = weakref.WeakKeyDictionary()

    def __set__(self, instance, value):
        self.values[instance] = int(value)

    def __get__(self, instance, onwer_class):
        if instance is None:
            return self
        else:
            return self.values.get(instance)

In [14]:
class Point:
    x = IntegerValue()

In [22]:
p = Point()
print(hex(id(p)))

0x7fd146122ab0


In [23]:
p.x = 100.1

In [24]:
p.x

100

In [25]:
Point.x.values.keyrefs()

[<weakref at 0x7fd145bea3e0; to 'Point' at 0x7fd146122ab0>]

In [26]:
del p

In [27]:
Point.x.values.keyrefs()

[]

In [36]:
class IntegerValue:
    def __init__(self):
        self.values = {}

    def __set__(self, instance, value):
        self.values[id(instance)] = int(value)

    def __get__(self, instance, onwer_class):
        if instance is None:
            return self
        else:
            return self.values.get(id(instance))

In [37]:
class Point:
    x = IntegerValue()

    def __init__(self, x):
        self.x = x

    def __eq__(self, other):
        return isinstance(other, Point) and self.x == other.x

In [38]:
p = Point(10.1)

In [39]:
p.x

10

In [40]:
p.x = 20.2

In [43]:
id(p), Point.x.values

(140536800479952, {140536800479952: 20})

In [44]:
import ctypes

def ref_count(address):
    return ctypes.c_long.from_address(address).value

In [45]:
p_id = id(p)

In [46]:
ref_count(p_id)

1

In [47]:
del p

In [48]:
ref_count(p_id)

100

In [49]:
Point.x.values

{140536800479952: 20}

In [50]:
p = Point(10.1)

In [51]:
weak_p = weakref.ref(p)

In [52]:
ref_count(id(p))

1

In [53]:
del p

In [54]:
weak_p

<weakref at 0x7fd145948e50; dead>